In [7]:
# 1) Importing libraries
from selenium import webdriver # Web browser automation for web scraping and automated testing.
from selenium.webdriver.firefox.service import Service # Browser Service Control for Selenium.
from webdriver_manager.firefox import GeckoDriverManager # Automatic Firefox driver management for Selenium.
from selenium.webdriver.firefox.options import Options as FirefoxOptions

from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup # HTML document analysis  for web scraping.

import time

In [ ]:
# 2) Set houses and apartments price range
lower_price = 400000
upper_price = 2000000

In [12]:
# 3) Web page URL from which the data will be extracted 
def get_page_url(page_number,rango_i,rango_s):
    
    base_page_url = "https://www.encuentra24.com/guatemala-es/bienes-raices-venta-de-propiedades-casas/guatemala-es-guatemala.{}?q=f_price.{}-{}|f_currency.GTQ|withcat.bienes-raices-venta-de-propiedades-casas,bienes-raices-venta-de-propiedades-apartamentos".format(page_number,rango_i,rango_s)
    
    return base_page_url

In [13]:
# 3) Create a driver object representing an automatically controlled Firefox web browser.
driver = webdriver.Firefox()
# Browser window maximize
driver.maximize_window()
# Go to the page want to Scrape
driver.get(url)